In [4]:
class ReactomePathway:
    def __init__(self, reactome_sID):
        self.is_Root= False
        self.is_Leaf= False
        self.name= ''
        self.reactome_sID= reactome_sID
        self.children= []
        self.parents= []
        self.level = -1
        self.root_id = ''
    def asdict(self):
        """ return the ReactomePathway  as dictionary 
        """
        return {'reactome_sID': self.reactome_sID, 'children': self.children, 'parents': self.parents, 'is_leaf': self.is_Leaf, 'is_root': self.is_Root, 'level': self.level, 'root_id': self.root_id}        
    def assert_leaf_root_state(self):
        if len(self.children) == 0:
            self.is_Leaf= True
        if len(self.parents) == 0:
            self.is_Root = True



In [5]:
class PathwayHierarchy(dict):
    def __init__(self, *arg, **kw):
       super(PathwayHierarchy, self).__init__(*arg, **kw)
    def add_hierarchy_levels(self):
        for k,v in self.items():
            level = 0
            not_at_root = True
            next_elem = k
            partent_name = ''
            while not_at_root:
                if len(self[next_elem].parents) < 1:
                    not_at_root = False
                    partent_name = next_elem
                    break
                else:
                    level += 1
                    next_elem = self[next_elem].parents[0] # [0] is temp fix disregards more than one parent
            v.level = level
            v.root_name = next_elem


    def hierarchyInfo(self):
        entries = len(self.keys()) 
        leafs = len([v for k,v in self.items() if v.is_Leaf])
        roots = len([v for k,v in self.items() if v.is_Root])
        return {'size': entries, 'leafs': leafs, 'roots': roots}
    def getLevel(self, key):
        print('test')


In [6]:
pathways = PathwayHierarchy()

organism = "MMU"

with open('ReactomePathwaysRelation.txt') as fh:
    for line in fh:
        line_list = line.strip().split('\t')
        left_entry = line_list[0]
        right_entry = line_list[1]
        if(organism in left_entry):
            if left_entry not in pathways.keys():
                pathways[left_entry] = ReactomePathway(left_entry)
                pathways[left_entry].children.append(right_entry)
            else:
                pathways[left_entry].children.append(right_entry)
            if right_entry not in pathways.keys():
                pathways[right_entry] = ReactomePathway(right_entry)
                pathways[right_entry].parents.append(left_entry)
            else:
                pathways[right_entry].parents.append(left_entry)

for k,v in pathways.items():
    v.assert_leaf_root_state()

pathways.add_hierarchy_levels()

In [7]:
print(pathways.hierarchyInfo())

{'size': 1704, 'leafs': 1127, 'roots': 27}


In [8]:
print(pathways['R-MMU-556833'].asdict())

{'reactome_sID': 'R-MMU-556833', 'children': ['R-MMU-1483257', 'R-MMU-400206', 'R-MMU-428157', 'R-MMU-74182', 'R-MMU-8848584', 'R-MMU-8957322', 'R-MMU-8964572', 'R-MMU-8978868', 'R-MMU-8979227', 'R-MMU-9018678'], 'parents': ['R-MMU-1430728'], 'is_leaf': False, 'is_root': False, 'level': 1, 'root-name': 'R-MMU-1430728'}


In [50]:
print("amount of multiparent-nodes", len([k for k,v in pathways.items() if len(v.parents) > 1]))

amount of multiparent-nodes 31
